In [6]:
import networkx as nx
import EoN
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from contact_tracing import *

import math
from constraint import ProbMinExposed
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ortools.linear_solver import pywraplp
from ortools.linear_solver.pywraplp import Constraint, Solver, Variable, Objective

from solve import *

from typing import Set, Dict
import pickle as pkl

In [2]:
G=grab_graph() #fetching the montgamery graph
n = len(G.nodes)
I0 = [i for i in range(n) if random.random() > 0.999]

In [3]:
N = G.number_of_nodes()

# COSTS = np.random.randint(1, 20, size=N)
COSTS = np.ones(N)
print(f"COSTS: {COSTS}")
# Compute distances
dist_dict = nx.multi_source_dijkstra_path_length(G, I0)

# convert dict vertex -> distance
# to distance -> [vertex]
level_dists = defaultdict(set)
for (i, v) in dist_dict.items():
    level_dists[v].add(i)

# print(level_dists)

# Obtain V_1 and V_2

# Set of vertices distance 1 away from infected I
V_1: Set[int] = level_dists[1]

# Set of vertices distance 2 away from infected I
V_2: Set[int] = level_dists[2]

COSTS: [1. 1. 1. ... 1. 1. 1.]


In [4]:
p1 = defaultdict(lambda: 1)
q = defaultdict(lambda: defaultdict(lambda : 1))

prob = ProbMinExposed(G,I0,V_1,V_2,p1,q,10,COSTS)

1
7794
1
11943
1
14488
1
16982
1
17157
1
19355
1
21314
1
36260
1
39800
1
43093
1
43373
1
54547
1
73229
1
7788
1
7795
1
17820
1
19030
1
19658
1
23275
1
24972
1
28578
1
30786
1
41729
1
55477
1
57639
1
57906
1
72314
1
7789
1
9349
1
11649
1
12222
1
13320
1
14167
1
15385
1
18700
1
20232
1
29125
1
35185
1
41573
1
59462
1
62049
1
11427
1
14483
1
24204
1
30549
1
45747
1
56048
1
59349
1
73447
1
7790
1
9664
1
10341
1
12994
1
15235
1
18748
1
21091
1
30022
1
30384
1
34046
1
48025
1
60407
1
7791
1
10389
1
12196
1
18535
1
24756
1
27760
1
30825
1
63111
1
69907
1
8567
1
10002
1
10503
1
12065
1
14458
1
24422
1
24470
1
29896
1
33442
1
35538
1
36029
1
43979
1
45327
1
55181
1
57580
1
70929
1
7792
1
8038
1
8524
1
17410
1
18430
1
20722
1
22871
1
27937
1
54193
1
55332
1
56686
1
7793
1
7964
1
9169
1
13521
1
21966
1
27441
1
31088
1
39632
1
45997
1
46412
1
48054
1
57369
1
70415
1
10908
1
11615
1
15116
1
16167
1
16675
1
22384
1
23223
1
23900
1
24228
1
26082
1
26357
1
27791
1
30422
1
31874
1
31901
1
52274
1
52389

49
40850
49
41221
49
41315
49
42480
49
53984
49
65142
49
73033
49
10302
49
13330
49
17349
49
18782
49
19635
49
25478
49
28190
49
31290
49
32321
49
33877
49
39777
49
41109
49
41186
49
41316
49
50805
49
52200
49
53124
49
60398
49
61430
49
64095
49
71835
49
8102
49
9381
49
9671
49
10471
49
11722
49
14844
49
17198
49
18257
49
18321
49
18783
49
19322
49
19765
49
21041
49
21733
49
22781
49
23942
49
25709
49
26140
49
27609
49
28945
49
31245
49
33293
49
33676
49
33993
49
36088
49
58777
49
59411
49
67039
49
70491
49
8103
49
8857
49
9510
49
9739
49
15464
49
16265
49
19369
49
19826
49
22424
49
24160
49
27698
49
39080
49
50670
49
57488
49
68819
49
69880
49
8104
49
10658
49
13065
49
15270
49
21094
49
21598
49
25258
49
25782
49
28998
49
30172
49
31772
49
34147
49
34920
49
34928
49
37319
49
39081
49
41215
49
43879
49
46530
49
51217
49
57165
49
62088
49
62597
49
65339
49
68240
49
69082
49
8105
49
9346
49
11571
49
11762
49
11792
49
12505
49
14856
49
18395
49
19192
49
20346
49
21773
49
23777
49
24628
49

9338
70
10768
70
10792
70
11604
70
19313
70
26140
70
26264
70
27219
70
31212
70
33676
70
42221
70
43098
70
51596
70
53629
70
8239
70
8873
70
9635
70
14344
70
17102
70
23140
70
24216
70
25397
70
26552
70
26749
70
27542
70
38206
70
42887
70
55780
70
64061
70
69675
70
69880
70
8240
70
8396
70
10658
70
15371
70
25372
70
25398
70
27146
70
30819
70
33519
70
34454
70
40059
70
53612
70
54213
70
54720
70
58624
70
59377
70
61251
70
64422
70
70211
70
70670
70
12505
70
30413
70
32204
70
32918
70
33425
70
43635
70
44371
70
47859
70
53979
70
60603
70
61153
70
68273
70
10930
70
11353
70
12883
70
12933
70
16964
70
18697
70
19571
70
21992
70
34363
70
37472
70
41370
70
48601
70
58827
70
63347
70
72947
73811
89
73811
296
73811
374
73811
744
73811
1064
73811
1513
73811
2012
73811
2036
73811
2378
73811
2430
73811
2631
73811
3242
73811
3964
73811
4194
73811
4361
73811
4414
73811
4508
73811
5419
73811
5731
73811
5812
73811
5856
73811
5860
73811
5872
73811
5916
73811
6110
73811
6116
73811
6397
73811
6519
7381

29262
145
30896
145
33139
145
45466
145
48186
145
49551
145
54737
145
57855
145
8748
145
15448
145
18784
145
19818
145
23008
145
23402
145
23843
145
24049
145
33302
145
45121
145
51416
145
53852
145
57940
145
59981
145
8749
145
14280
145
18403
145
20570
145
34636
145
35736
145
37075
145
39081
145
42642
145
54221
145
55670
145
57626
145
61812
145
62995
145
63960
145
10553
145
12383
145
17109
145
20637
145
22017
145
24576
145
25498
145
26330
145
28014
145
30621
145
31626
145
36248
145
38916
145
52450
145
57001
145
60487
145
8750
145
9326
145
10998
145
16771
145
18343
145
20248
145
21154
145
23148
145
23890
145
31928
145
33812
145
39572
145
40522
145
43581
145
43678
145
61032
145
66765
145
70406
145
73830
57502
426
57502
485
57502
722
57502
742
57502
776
57502
889
57502
991
57502
1015
57502
1410
57502
1512
57502
1541
57502
1744
57502
1930
57502
2170
57502
2542
57502
2617
57502
2916
57502
3480
57502
4792
57502
5026
57502
5586
57502
5746
57502
5838
57502
5867
160
8860
160
9521
160
11231
160

229
15634
229
31844
229
32983
229
34775
229
35019
229
59061
229
71184
229
13247
229
17286
229
25397
229
26721
229
30386
229
44981
229
59062
229
64061
229
64276
229
66399
229
10560
229
11492
229
12714
229
16436
229
17130
229
17598
229
22694
229
23338
229
28786
229
33150
229
34125
229
35599
229
36524
229
44496
229
54971
229
58587
229
62570
229
64422
229
68067
229
68206
229
73914
229
10886
229
11206
229
13494
229
16936
229
22918
229
22976
229
32445
229
32918
229
52177
229
52378
229
53493
229
55962
229
57268
229
57296
229
58198
229
71602
229
72045
229
9339
229
9818
229
10930
229
16410
229
16787
229
21992
229
24770
229
35209
229
39701
229
39780
229
57739
229
58468
229
59855
229
61715
229
62481
229
63347
229
64927
229
66775
229
68877
229
73579
230
9347
230
14174
230
16421
230
20285
230
61762
230
9341
230
9348
230
9982
230
13097
230
21148
230
22543
230
23283
230
30168
230
33603
230
59733
230
72333
230
10293
230
18763
230
27832
230
29156
230
36233
230
39713
230
40889
230
58518
230
9342
230
964

...

73479
5825
73804
5825
74308
5825
74498
5825
57111
5825
57549
5825
59071
5825
59961
5825
69527
5825
70792
5825
71525
5825
72439
5825
72923
5825
73428
5825
57112
5825
59813
5825
61354
5825
71391
5825
73142
5825
73481
5825
73508
5825
73688
5825
73717
5825
59709
5825
57114
5825
57965
5825
59180
5825
71076
5825
73403
5825
74520
5825
57115
5825
58613
5825
68195
5825
71216
5825
72265
5825
72664
5825
73835
5825
73930
5825
74521
5825
60028
5825
65617
5825
69946
5825
71767
5825
72622
5825
72700
5825
57116
5825
59939
5825
68701
5825
73377
5825
57117
5825
59493
5825
60285
5825
71167
5825
71903
5825
72446
5825
73091
5825
73820
5825
73875
5825
74190
5825
59166
5825
67822
5825
68335
5825
69588
46786
516
46786
3420
46786
3591
46786
3732
46786
3773
46786
3826
46786
4037
46786
4053
46786
4083
46786
4180
46786
4197
46786
4652
54992
88
54992
156
54992
250
54992
305
54992
326
54992
371
54992
480
54992
662
54992
706
54992
721
54992
818
54992
872
54992
1936
54992
5581
14035
111
14035
599
14035
757
5846
57307

46998
3528
46998
3575
46998
3603
46998
3757
46998
4230
46998
4477
46998
4520
46998
4560
46998
4577
6037
68243
6037
68995
6037
69077
6037
59620
6037
60097
6037
62971
6037
67981
6037
59058
6037
60396
6037
62620
6037
64225
6037
65099
6037
65902
6037
66362
6037
59059
6037
59160
6037
60562
6037
67558
6037
69988
6037
63966
6037
66499
6037
67441
6037
67684
6037
59062
6037
59482
6037
62415
6037
67725
6037
70202
6037
61296
6037
59345
6037
59618
6037
60957
6037
61715
6037
62481
6037
67102
6037
73579
6071
59416
6071
63536
6071
61353
6071
72314
6071
72839
6071
73498
6071
59348
6071
60201
6071
74248
6071
74475
6071
62909
6071
65404
6071
69043
6071
59350
6071
60007
6071
61537
6071
69717
6071
74074
6071
59351
6071
60222
6071
65493
6071
65905
6071
66078
6071
59352
6071
70413
6071
70929
6071
72320
6071
73468
6071
59353
6071
59836
6071
60612
6071
61380
6071
72570
6071
61042
6071
69722
6071
71779
6071
73882
6074
61652
6074
66811
6074
59373
6074
72882
6074
61385
6074
63501
6074
70438
6074
59375
6074
59735

64028
6558
64045
6558
64210
6558
64286
6558
64345
6558
64431
6558
64555
6558
64831
6558
65349
6558
65636
6558
65994
6558
63734
6558
63750
6558
64089
6558
64471
6558
64823
6558
65549
6558
70719
6558
63735
6558
63998
6558
64038
6558
64399
6558
64564
6558
64633
6558
65235
6558
66749
63908
109
63908
1185
63908
1901
63908
2020
63908
3642
63908
5624
63908
5837
63908
6220
6576
63884
6576
63947
6576
65586
6576
63877
6576
63885
6576
64111
6576
67710
6576
70800
6576
65175
6576
67408
6576
63878
6576
64350
6576
65526
6576
63879
6576
64442
6576
73153
6576
63880
6576
63915
6576
65067
6576
67285
6576
63881
6576
64116
6576
64832
6576
63883
6576
66633
6582
63938
6582
66785
6582
63939
6582
64558
6582
64884
6582
65479
6582
66662
6582
67035
6582
73859
6582
63932
6582
63940
6582
73626
6582
63933
6582
65429
6582
63934
6582
67593
6582
64292
6582
65086
6582
66516
6582
63935
6582
65776
6582
64431
6582
63936
6582
63946
31169
75
31169
198
31169
328
31169
534
31169
584
31169
644
31169
662
31169
705
31169
716
3116

7142
69221
7142
70628
31719
42
31719
355
31719
436
31719
574
31719
714
31719
730
31719
1036
31719
1510
31719
1542
31719
1769
31719
1830
31719
2043
31719
2067
31719
2255
31719
2288
31719
2431
31719
2511
31719
2562
31719
2585
31719
2600
31719
2673
31719
2693
31719
2775
31719
2862
7143
68858
7143
69276
7143
69492
7143
69686
7143
71064
7143
71796
7143
72279
7143
73214
7143
69875
7143
70165
7143
70398
7143
70836
7143
71498
7143
68851
7143
68943
7143
69174
7143
69867
7143
70224
7143
71260
7143
71666
7143
71763
7143
71861
7143
69034
7143
69253
7143
69297
7143
70225
7143
70263
7143
71024
7143
71298
7143
72002
7143
68862
7143
69353
7143
70007
7143
70658
7143
71252
7143
73258
7143
69290
7143
69655
7143
69682
7143
70026
7143
70449
7143
72116
7143
68854
7143
69928
7143
70266
7143
70320
7143
70450
7143
71563
7143
71600
7143
68855
7143
68920
7143
69420
7143
70643
7143
71521
7143
71549
7143
71801
7143
68914
7143
69604
7143
70203
7143
70590
7143
68857
7143
69968
7143
70557
7143
70628
7143
70931
7143
7

74237
23910
662
23910
705
23910
716
23910
731
23910
987
23910
1116
23910
1185
23910
1291
23910
1296
23910
1388
23910
1613
23910
1738
23950
744
23950
1513
23950
2012
56737
4294
56737
4657
56737
5423
56737
5648
23974
303
23974
316
23974
377
23974
434
23974
662
23974
705
23974
716
23974
731
23974
783
23974
794
23974
848
23974
880
23974
895
23974
1058
23974
1116
23974
1185
23974
1291
23974
1296
23974
1298
23974
1303
23974
1338
23974
1388
23974
1409
23974
1655
23974
1738
48571
516
48571
530
48571
705
48571
3160
48571
3420
48571
3591
48571
3732
48571
3773
48571
3826
48571
4037
48571
4053
48571
4083
48571
4180
48571
4197
48571
4305
48571
4652
48571
4721
48571
4837
48571
4859
15807
166
15807
1023
73151
3383
73151
3511
73151
3632
73151
3766
73151
3783
73151
3800
73151
4051
73151
4077
73151
4304
73151
4786
73151
4968
73151
4971
73151
5003
73151
5055
73151
5318
73151
5324
73151
5688
73151
7625
48583
3338
48583
3620
48583
3879
48583
3921
48583
4016
48583
4021
48583
4114
48583
4148
48583
4191
48583

In [5]:
prob.solve_lp()
probabilities = np.array(prob.getVariables())

In [7]:
iterated_round(prob, 11)

IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
"""STOP HERE"""

In [ ]:
len(V_1)

In [ ]:
V1 = set([])
for i in I0:
    for j in G.neighbors(i):
        if j not in I0:
            V1.add(j)


In [ ]:
len(Q['v'].unique())

In [ ]:
len(I0)

In [ ]:
len(V_2)

In [ ]:
P,Q = PQ(G,I0,runs = 20)

In [ ]:
for u in V1:
    for v in G.neighbors(u):
        if v in V2:
            print(u)
            print(v)
            #coeff = (q[u][v] * self.p1[u])
            #print(f"Coeff: {coeff}")
            #self.solver.Add(self.Y2[v] >= coeff * self.Y1[u])

In [ ]:
G.adj[1076]

In [ ]:
Q[Q.u == 1076]

In [ ]:
prob.getVariables()

In [ ]:
#test simplify

a = 1
b = 0

total_a = 0
total_b = 0

n=1000000

for i in range(n):
    (x,y,new_a,new_b) = simplify(a,b)
    
    if x==-1:
        total_a += new_a
        total_b += y
    elif y==-1:
        total_a += x
        total_b += new_b
    else:
        total_a += x
        total_b += y

print(str(total_a/n) + " " + str(a))
print(str(total_b/n) + " " + str(b))

In [ ]:
#test D
p = [.5,.5,.25,.75]

n=10000
total = np.zeros(4)

for i in range(n):
    a = D(p)
    if np.sum(a) != np.sum(p):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)

In [ ]:
#test D_prime
p = [.5,.4,.25,.75]

n=5000
total = np.zeros(4)

for i in range(n):
    a = D(p)
    if (np.sum(a) != math.floor(np.sum(p))) & (np.sum(a) != math.ceil(np.sum(p))):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)